In [1]:
import sys
from torch.nn.utils.rnn import pad_sequence
import transformers
import torch
# sys.path.append("./Incontext-learning") # this part works for goole colab

In [2]:
import utility
import metric
import importlib
importlib.reload(utility)
importlib.reload(metric)
from utility import (data_selection, evaluate_zeroshot,evaluate_finetuning,evaluate_demonstrations as  evaluate_fewshots, finetune_model_eval, extract_attentionweights, extract_raw_attention_scores_simple)
from metric import Rec2FTP, SimAOU
from torch.utils.data import DataLoader


In [3]:
from transformers.models.qwen3 import modeling_qwen3
from customized_Qwen import Qwen3Attention_v1
modeling_qwen3.Qwen3Attention = Qwen3Attention_v1

# check point

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load SST-2 dataset
# dataset = load_dataset("glue", "sst2")

# load SST-5 dataset
dataset = load_dataset("SetFit/sst5")

# Load Qwen3 tokenizer and model
model_name = "Qwen/Qwen3-0.6B"
model_path = "/home/students/wli/UniHeidelberg/semster2/final_projects/models/Qwen3-0.6B-Base"
# model_path = model_name
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map = "auto")

tokenizer.padding_side = "left"
# Make sure tokenizer has pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# if getattr(model.config.pad_token_id, None) is None:
    # pass

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
# import Qwne3ForCausalLM
from transformers import Qwen3ForCausalLM
model = Qwen3ForCausalLM.from_pretrained(model_path, device_map = "auto")

In [6]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention_v1(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layernorm): 

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2210
    })
})

In [6]:
output = extract_attentionweights(
    model, tokenizer, ["This is a test sentence."," Another test sentence. label: "], 
    batch_size=2, before_softmax=True
)

Found 28 layers in model.model.layers
Registering hook on layer 0: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 1: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 2: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 3: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 4: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 5: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 6: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 7: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 8: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering hook on layer 9: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Registering 

  0%|          | 0/1 [00:00<?, ?it/s]

Processing batch 0 with 2 samples
Layer 0: Module type: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Layer 0: Input type: <class 'tuple'>, Input length: 0
Layer 0: No valid input found
Layer 1: Module type: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Layer 1: Input type: <class 'tuple'>, Input length: 0
Layer 1: No valid input found
Layer 2: Module type: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Layer 2: Input type: <class 'tuple'>, Input length: 0
Layer 2: No valid input found
Layer 3: Module type: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Layer 3: Input type: <class 'tuple'>, Input length: 0
Layer 3: No valid input found
Layer 4: Module type: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Layer 4: Input type: <class 'tuple'>, Input length: 0
Layer 4: No valid input found
Layer 5: Module type: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3Attention'>
Layer 5: Input typ

100%|██████████| 1/1 [02:52<00:00, 172.41s/it]

All hooks removed


In [17]:
output = extract_raw_attention_scores(
    model, tokenizer, ["This is a test sentence."," Another test sentence. label: "], 
    batch_size=1
)

  0%|          | 0/2 [01:06<?, ?it/s]


IndexError: tuple index out of range

In [8]:
train_dataset = dataset["train"] # the orginal paper use this
test_dataset = dataset["validation"]

def add_idx(example, idx):
    example["idx"] = idx
    return example

test_dataset = test_dataset.map(add_idx, with_indices=True)
# Format examples as causal LM inputs
def preprocess_function(examples):
    label_map = {0: "negative", 1: "positive"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["sentence"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

def preprocess_function_sst5(examples):
    label_map = {0: "terrible", 1: "bad", 2: "neutral", 3: "good", 4: "very great"}
    inputs = [
        f"Sentence: {sentence} Label: {label_map[label]}"
        for sentence, label in zip(examples["text"], examples["label"])
    ]
    # Tokenize with padding/truncation
    tokenized = tokenizer(
        inputs,
        truncation=True,
        padding=False,
        max_length=256,
        # return_tensors="pt"
        return_tensors=None
    )
    # Set labels equal to input_ids for causal LM loss
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


# Tokenize the dataset
# tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
train_tokenized_datasets = train_dataset.map(preprocess_function_sst5, batched = True, remove_columns = train_dataset.column_names)
test_tokenized_datasets = test_dataset.map(preprocess_function_sst5, batched=True, remove_columns=test_dataset.column_names)
# Data collator (handles padding dynamically in batch)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [10]:
# test the behavior of LLM before fine tuing
prompt = "Sentence: a stirring , funny and finally transporting re-imagining of beauty and the beast and 1930s horror films. Label:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# You can control max_length or early stopping
output_ids = model.generate(
    **inputs,
    max_new_tokens=10,    # adjust based on expected label length
    do_sample=False,       # greedy decoding
    pad_token_id=tokenizer.pad_token_id
)

# Decode generated tokens
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(generated_text)

customized attention layer 0 is called
customized attention layer 1 is called
customized attention layer 2 is called
customized attention layer 3 is called
customized attention layer 4 is called
customized attention layer 5 is called
customized attention layer 6 is called
customized attention layer 7 is called
customized attention layer 8 is called
customized attention layer 9 is called
customized attention layer 10 is called
customized attention layer 11 is called
customized attention layer 12 is called
customized attention layer 13 is called
customized attention layer 14 is called
customized attention layer 15 is called
customized attention layer 16 is called
customized attention layer 17 is called
customized attention layer 18 is called
customized attention layer 19 is called
customized attention layer 20 is called
customized attention layer 21 is called
customized attention layer 22 is called
customized attention layer 23 is called
customized attention layer 24 is called
customized

In [10]:
# test_dataset = test_dataset.select(range(32)) # use a smaller test set for debugging

In [11]:
for batch in DataLoader(test_dataset, batch_size=2, shuffle=True):
    print(batch)
    print(type(batch))
    break

{'text': ["although there are several truly jolting scares , there 's also an abundance of hackneyed dialogue and more silly satanic business than you can shake a severed limb at .", "it believes it 's revealing some great human truths , when , in reality , it 's churning ground that has long passed the point of being fertile ."], 'label': tensor([2, 2]), 'label_text': ['neutral', 'neutral'], 'idx': tensor([836, 911])}
<class 'dict'>


In [12]:

a = SimAOU(model,tokenizer,train_dataset, test_dataset, best_seed = 0, num_data_points = 32, data_type = "sst5", batch_size_icl = 32, batch_size_zsl = 16)

  0%|          | 0/138 [00:00<?, ?it/s]

RAM usage: 2303.54 MB


  1%|          | 1/138 [00:09<21:39,  9.48s/it]

RAM usage: 2333.88 MB


  1%|▏         | 2/138 [00:18<20:37,  9.10s/it]

...

 99%|█████████▉| 137/138 [01:20<00:00,  1.60it/s]

RAM usage: 11680.49 MB


100%|██████████| 138/138 [01:21<00:00,  1.70it/s]

RAM usage: 11680.49 MB


SimAOU(FT) score of  data type sst5 is 0.2837 
 SimAOU(Random) score of  data type sst5 is -0.0000


In [14]:
a

(0.2836596573028585, -2.7076809664290994e-06)

In [13]:
# finetune_model_eval(model, tokenizer,train_dataset,test_dataset, best_seed = 50, data_type="sst5", num_data_points = 32)